In [1]:
import numpy as np
import pandas as pd
import dias.rewriter

In [2]:
df = pd.read_csv("./datasets/spscientist__student-performance-in-exams__StudentsPerformance.csv")

# -- STEFANOS -- Replicate Data

In [3]:
factor = 1000
df = pd.concat([df]*factor)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   gender                       1000000 non-null  object
 1   race/ethnicity               1000000 non-null  object
 2   parental level of education  1000000 non-null  object
 3   lunch                        1000000 non-null  object
 4   test preparation course      1000000 non-null  object
 5   math score                   1000000 non-null  int64 
 6   reading score                1000000 non-null  int64 
 7   writing score                1000000 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 68.7+ MB


# -- STEFANOS -- Prelude

In [4]:
passmark = 40

In [5]:
df['Math_PassStatus'] = np.where(df['math score']<passmark, 'F', 'P')

In [6]:
df['Reading_PassStatus'] = np.where(df['reading score']<passmark, 'F', 'P')

In [7]:
df['Writing_PassStatus'] = np.where(df['writing score']<passmark, 'F', 'P')

# -- STEFANOS -- Apply Vectorized

*Original time*: 5.16s

*Rewritten time*: 113ms

## Speedup: 45x

In [8]:
%%time
%%rewrite
df['OverAll_PassStatus'] = df.apply(lambda x : 'F' if x['Math_PassStatus'] == 'F' or 
                                    x['Reading_PassStatus'] == 'F' or x['Writing_PassStatus'] == 'F' else 'P', axis =1)

--------- Final Source -----------
def _REWR_apply_vec(df):
    if type(df) == pd.DataFrame:
        return pd.Series(np.select([(df['Math_PassStatus'] == 'F') | (df[
            'Reading_PassStatus'] == 'F') | (df['Writing_PassStatus'] ==
            'F')], ['F'], default='P'))
    else:
        return df.apply(lambda x: 'F' if x['Math_PassStatus'] == 'F' or x[
            'Reading_PassStatus'] == 'F' or x['Writing_PassStatus'] == 'F' else
            'P', axis=1)
df['OverAll_PassStatus'] = _REWR_apply_vec(df)

CPU times: user 118 ms, sys: 11.6 ms, total: 130 ms
Wall time: 129 ms
